# OpenMC Learning
This notebook is for me to learn how to use OpenMC. The first problem is a sphere of radius 20 cm 
made of U-235 with 10,000 particles to calculate the flux at the surface of the sphere.

In [28]:
import numpy as np
import matplotlib.pyplot
import openmc

## Problem 1

In [29]:
#Add Fe-56
materials = openmc.Materials()

u235 = openmc.Material()
u235.add_nuclide('U235', 1, 'ao')
u235.set_density('g/cm3', 19.1)
u235.temperature = 900

materials.append(u235)

In [30]:
# Defining geometry
geometry = openmc.Geometry()

sphere = openmc.Sphere(r=20)
inside_sphere = -sphere
outside_sphere = +sphere
sphere.boundary_type = 'vacuum'

cell1 = openmc.Cell(fill=u235, region=inside_sphere)
cell2 = openmc.Cell(region=outside_sphere)
universe = openmc.Universe(cells=[cell1, cell2])

geometry.root_universe = universe

In [33]:
settings = openmc.Settings()
settings.run_mode = 'eigenvalue'
settings.particles = 10000
settings.generations_per_batch = 10
settings.batches = 100
settings.inactive = 5

In [35]:
tally = openmc.Tally(1)
tally.filters = [openmc.CellFilter(cell1)]
tally.scores = ['total']
tallies = openmc.Tallies([tally])

In [36]:
materials.export_to_xml()
geometry.export_to_xml()
settings.export_to_xml()
tallies.export_to_xml()

openmc.run()

                                %%%%%%%%%%%%%%%
                           %%%%%%%%%%%%%%%%%%%%%%%%
                        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                      %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                                     %%%%%%%%%%%%%%%%%%%%%%%%
                 ###############      %%%%%%%%%%%%%%%%%%%%%%%%
                ##################     %%%%%%%%%%%%%%%%%%%%%%%
                ###################     %%%%%%%%%%%%%%%%%%%%%%%
                ####################     %%%%%%%%%%%%%%%%%%%%%%
                #####################     %%%%%%%%%%%%%%%%%%%%%
                ######################     %%%%%%%%%%%%%%%%%%%%
                #######################     %%%%%%%%%%%%%%%%%%
                 #######################     %%%%%%%%%%%%%%%%%
                 #####################

RuntimeError: No cross_sections.xml file was specified in materials.xml or in the OPENMC_CROSS_SECTIONS environment variable. OpenMC needs such a file to identify where to find data libraries. Please consult the user's guide at https://docs.openmc.org/ for information on how to set up data libraries. -------------------------------------------------------------------------- MPI_ABORT was invoked on rank 0 in communicator MPI_COMM_WORLD with errorcode -1. NOTE: invoking MPI_ABORT causes Open MPI to kill all MPI processes. You may or may not see output from other processes, depending on exactly when Open MPI kills them. --------------------------------------------------------------------------

## Problem 2
This problem performs depletion

In [37]:
import openmc.deplete as od

In [38]:
model = openmc.Model(geometry = geometry, materials = materials, settings = settings)

In [39]:
microsxs = od.MicroXS.from_model(model, u235, chain_file="chain_simple.xml")
ind_op = od.IndependentOperator(materials, microxs, chain_file = "chain_simple.xml")

OSError: Error reading file 'chain_simple.xml': failed to load external entity "chain_simple.xml"